In [30]:
import pyspark
import findspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import StructType, StructField, IntegerType
from pyspark.sql.functions import avg, col, mean, avg, countDistinct, desc, split
import pyspark.sql.functions

In [2]:
findspark.init()
findspark.find()
conf = pyspark.SparkConf().setAppName("hendrik_app").setMaster('local[17]')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)
sql_c = SQLContext(sc)

df = sql_c.read.csv('/data/dataprocessing/interproscan/all_bacilli.tsv',header=False, sep="\t")
df = df.toDF('protein_accession', 'sequence', 'sequence_length', 
'analysis', 'signature_accession', 'signature_description', 'start', 'stop', 
'score', 'status', 'date',  'interPro_annot_accession', 'interPro_annot_description', 
'GO_annotations', 'Pathways_annotations' 
)

22/06/17 12:41:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df.show()

+--------------------+--------------------+---------------+---------------+-------------------+---------------------+-----+----+-----------+------+----------+------------------------+--------------------------+--------------------+--------------------+
|   protein_accession|            sequence|sequence_length|       analysis|signature_accession|signature_description|start|stop|      score|status|      date|interPro_annot_accession|interPro_annot_description|      GO_annotations|Pathways_annotations|
+--------------------+--------------------+---------------+---------------+-------------------+---------------------+-----+----+-----------+------+----------+------------------------+--------------------------+--------------------+--------------------+
|gi|29898682|gb|AA...|92d1264e347e14924...|            547|        TIGRFAM|          TIGR03882| cyclo_dehyd_2: ba...|    2| 131|    1.6E-21|     T|25-04-2022|               IPR022291|      Bacteriocin biosy...|                   -|          

In [4]:
df = df.filter(df['GO_annotations'] != '-')

In [5]:
df.columns[11]

'interPro_annot_accession'

In [6]:
df.select('interPro_annot_accession').show()

+------------------------+
|interPro_annot_accession|
+------------------------+
|               IPR001867|
|               IPR001867|
|               IPR001867|
|               IPR001867|
|               IPR001789|
|               IPR001789|
|               IPR001789|
|               IPR000551|
|               IPR000551|
|               IPR000551|
|               IPR004636|
|               IPR005814|
|               IPR015421|
|               IPR005814|
|               IPR005814|
|               IPR005814|
|               IPR005814|
|               IPR005814|
|               IPR015422|
|               IPR004636|
+------------------------+
only showing top 20 rows



Question 1:

In [7]:
unique_GOs = len(df.select('interPro_annot_accession').distinct().collect())
unique_GOs = df.select('interPro_annot_accession').distinct()
print(unique_GOs._sc._jvm.PythonSQLUtils.explainString(unique_GOs._jdf.queryExecution(), 'simple'))
# unique_GOs

== Physical Plan ==
*(2) HashAggregate(keys=[interPro_annot_accession#57], functions=[])
+- Exchange hashpartitioning(interPro_annot_accession#57, 200), ENSURE_REQUIREMENTS, [id=#92]
   +- *(1) HashAggregate(keys=[interPro_annot_accession#57], functions=[])
      +- *(1) Project [_c11#27 AS interPro_annot_accession#57]
         +- *(1) Filter (isnotnull(_c13#29) AND NOT (_c13#29 = -))
            +- FileScan csv [_c11#27,_c13#29] Batched: false, DataFilters: [isnotnull(_c13#29), NOT (_c13#29 = -)], Format: CSV, Location: InMemoryFileIndex[file:/data/dataprocessing/interproscan/all_bacilli.tsv], PartitionFilters: [], PushedFilters: [IsNotNull(_c13), Not(EqualTo(_c13,-))], ReadSchema: struct<_c11:string,_c13:string>




Question 2: ( wrong )

In [8]:
GO_counts = df.groupBy('interPro_annot_accession').count().sort('count')
# # GO_counts_list = [i for i in range(len(GO_counts))]
# GO_counts_list = []

# for i in range(df.count()):
#     count = GO_counts.orderBy(desc('count')).take(df.count())[2][i]
#     GO_counts_list.append(count)
    

Question 3:

In [9]:
Day_count = df.groupby(df.columns[13]).count()
Day_count.orderBy(desc('count')).take(10)[0][0]

'GO:0003700|GO:0006355'

Question 4: ?

In [10]:
df = df.withColumn('diff_length', ( df["stop"] - df["start"]) / df['sequence_length'])

GO_length = df.groupBy('interPro_annot_accession').mean('diff_length')
GO_length.columns
# GO_length.select('avg(diff_length)').show()
GO_length.agg({'avg(diff_length)': 'mean'}).show()

+---------------------+
|avg(avg(diff_length))|
+---------------------+
|   0.6347509732551729|
+---------------------+



Question 5:

In [11]:
question5 = df.groupby(df['interPro_annot_accession']).count()
res=[]
for i in question5.orderBy(desc('count')).take(10):
    res.append(i[0])
res

['IPR003439',
 'IPR000515',
 'IPR001789',
 'IPR000182',
 'IPR004358',
 'IPR000835',
 'IPR000524',
 'IPR001647',
 'IPR001867',
 'IPR017871']

Question 6: If you select InterPRO features that are almost the same size (within 90-100%) as the protein itself, what is the top10 then?

In [12]:
# df = df.withColumn('diff_length', (df["stop"] - df["start"]) / df['sequence_length']) 

In [13]:
question6 = df.filter(df['diff_length'] >= 0.90)
# question6.show()
# df['interPro_annot_accession']
res=[]
question6.orderBy(desc('diff_length')).take(10)
for i in question6.orderBy(desc('diff_length')).take(10):
    res.append(i[11])
res

['IPR006308',
 'IPR006308',
 'IPR006308',
 'IPR006308',
 'IPR006308',
 'IPR006308',
 'IPR006308',
 'IPR006308',
 'IPR006308',
 'IPR006308']

Question 7:

In [28]:
df.columns[12]

'interPro_annot_description'

In [43]:
# df.select('interPro_annot_description').show(1)
question7 = split(df['interPro_annot_description'],",",limit=None)
question7.getItem(1)

TypeError: split() missing 1 required positional argument: 'pattern'

In [60]:
df = df.withColumn('sep_column', df.select(split(df['interPro_annot_description'], "\\s").alias('str')))
df

AssertionError: col should be Column